In [103]:
import os
import json
import pandas as pd
import plotly.graph_objects as go

In [104]:
data_dir = './data_dump'
pd.set_option('display.max_columns', None)

In [106]:
# for fn in os.listdir(data_dir):
#   with open(os.path.join(data_dir, fn), 'r') as f:
#     data = json.load(f)
#     df_raw = pd.json_normalize(data)
with open(os.path.join(data_dir, '20250725-172602_proj_dzpYvLgn.json'), 'r') as f:
  data = json.load(f)
  df_raw = pd.json_normalize(data)

In [107]:
static_data = {}
static_cols = ['angular_velocity', 'position', 'velocity', 'rotation']
for col in df_raw.columns:
  if len(df_raw[col].unique()) == 1 and all(stc not in col for stc in static_cols):
    static_data[col] = df_raw[col].unique()[0]
df = df_raw.drop(list(static_data.keys()), axis=1)
static_data

{'medium.base_density': np.float64(1.225),
 'medium.drag_coef': np.float64(0.47),
 'medium.drag_model': 'quadratic',
 'medium.molar_mass': np.float64(0.029),
 'medium.rha_coef': np.float64(1e-05),
 'medium.speed_of_sound': np.float64(343.0),
 'medium.type': 'gas',
 'medium.viscosity': np.float64(1.81e-05),
 'naming.ammo_name': '5.56x45_M855',
 'naming.projectile_uid': 'proj_dzpYvLgn',
 'naming.weapon_name': 'SCAR_L_CQC',
 'projectile.ammo': '<Resource#-9223371996170025611>',
 'projectile.caliber': np.float64(5.7),
 'projectile.core_caliber': np.float64(5.4),
 'projectile.core_hardness': np.float64(0.6),
 'projectile.core_mass': np.float64(2.0),
 'projectile.cross_section': np.float64(2.55175863287831e-05),
 'projectile.drag_coef': np.float64(0.28),
 'projectile.fragmentation_chance': np.float64(0.1),
 'projectile.fragmentation_count': np.int64(0),
 'projectile.fragments_max': np.int64(4),
 'projectile.fragments_min': np.int64(2),
 'projectile.impact_count': np.int64(0),
 'projectile.ki

In [108]:
df['projectile.velocity.length'] = (df['projectile.velocity.x']**2 + df['projectile.velocity.y']**2 + df['projectile.velocity.z']**2)**0.5
df['projectile.position.length'] = (df['projectile.position.x']**2 + df['projectile.position.y']**2 + df['projectile.position.z']**2)**0.5
df.columns

Index(['projectile.angular_velocity.x', 'projectile.angular_velocity.y',
       'projectile.angular_velocity.z', 'projectile.current_drag_coef',
       'projectile.effective_cross_section', 'projectile.flight_time',
       'projectile.mach_number', 'projectile.position.x',
       'projectile.position.y', 'projectile.position.z',
       'projectile.rotation.x.x', 'projectile.rotation.x.y',
       'projectile.rotation.x.z', 'projectile.rotation.y.x',
       'projectile.rotation.y.y', 'projectile.rotation.y.z',
       'projectile.rotation.z.x', 'projectile.rotation.z.y',
       'projectile.rotation.z.z', 'projectile.state', 'projectile.velocity.x',
       'projectile.velocity.y', 'projectile.velocity.z', 'timing.exec_ratio',
       'timing.exec_time', 'timing.frame', 'timing.time',
       'projectile.velocity.length', 'projectile.position.length'],
      dtype='object')

In [113]:
fig = go.Figure()

marker_step = 10
marker_indices = df.index[::marker_step]
mach_min = df['projectile.mach_number'].min()
mach_max = df['projectile.mach_number'].max()
mach_threshold = 1.0
mach_threshold_norm = (mach_threshold - mach_min) / (mach_max - mach_min)
mach_colorscale = [
  [0.0, '#2f9'],
  [mach_threshold_norm, '#4af'],
  [mach_threshold_norm, '#f00'],
  [1.0, '#ff0']
]

fig.add_trace(go.Scatter3d(
  x=df['projectile.position.x'],
  z=df['projectile.position.y'],
  y=df['projectile.position.z'],
  mode='markers',
  marker=dict(
    size=3,
    color=df['projectile.current_drag_coef'][2:],
    colorscale='Jet',
    colorbar=dict(
      title='Drag Coefficient',
      y=0.0,
      len=0.5
    )
  )
))
fig.add_trace(go.Scatter3d(
  x=df.loc[marker_indices, 'projectile.position.x'],
  z=df.loc[marker_indices, 'projectile.position.y'],
  y=df.loc[marker_indices, 'projectile.position.z'],
  mode='lines',
  line=dict(
    width=5,
    color=df['projectile.mach_number'],
    colorscale=mach_colorscale,
    colorbar=dict(
      title='Mach Number',
      y=0.5,
      len=0.5
    ),
  ),
  name='Position',
  customdata=pd.concat([
    df[[
    'timing.frame',
    'projectile.flight_time',
    'projectile.velocity.z',  # Rate Z
    'projectile.velocity.y',  # Rate Y
    'projectile.velocity.x',  # Rate X
    'projectile.velocity.length',  # Vel m/s
    'projectile.mach_number',      # Vel M
    'projectile.current_drag_coef' # Cd
  ]],
    df[[
      'projectile.position.z',  # Dist (Z)
      'projectile.position.y',  # Drop (Y)
      'projectile.position.x',  # Drift (X)
    ]].abs()
  ], axis=1),
  hovertemplate=(
    "Frame %{customdata[0]} (%{customdata[1]:.5f}s)<br>" +
    "Dist: %{customdata[8]:.2f} m | Drop: %{customdata[9]:.2f} m | Drift: %{customdata[10]:.2f} m<br>" +
    "Rate: (%{customdata[2]:.2f}, %{customdata[3]:.2f}, %{customdata[4]:.2f}) m/s<br>" +
    "Velocity: %{customdata[5]:.2f} m/s (%{customdata[6]:.2f} M)<br>" +
    "Cd: %{customdata[7]:.3f}" +
    "<extra></extra>"
  )
))


scene_ax = dict(
  backgroundcolor='#444',
  gridcolor='#666',
  showbackground=True,
  zerolinecolor='#666',
  color='white'
)
fig.update_layout(
  paper_bgcolor='#333',
  plot_bgcolor='#444',
  font=dict(
    color='white'
  ),
  scene_camera=dict(
    eye=dict(x=-1.5, y=0.0, z=0.0)  # положение "глаза камеры"
  ),
  scene=dict(
    aspectmode='manual',
    aspectratio=dict(x=1.0, y=2.5, z=0.5),
    xaxis=scene_ax,
    yaxis=scene_ax,
    zaxis=scene_ax
  ),
  margin=dict(l=0, r=0, b=0, t=10)
)